In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 60)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정
SEED = 42

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DACON/LG/train.csv")
df

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,...,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,0.0,4.34,0.18,13.34,13.37,13.40,13.52,13.44,3.11,3.17,3.06,3.13,1,2.05,2.07,2.05,2.06,2.06,2.35,...,21.20,20.99,21.28,21.09,0.29,1463,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,0.0,4.38,0.18,13.33,13.33,13.46,13.51,13.42,2.97,3.11,2.91,3.20,1,2.10,2.10,2.10,2.10,2.17,2.28,...,21.16,21.03,21.16,21.13,0.13,1463,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,0.0,4.36,0.15,13.36,13.36,13.39,13.51,13.43,3.04,3.04,3.01,3.12,1,2.07,2.06,2.04,2.07,2.10,2.26,...,21.13,21.03,21.17,21.12,0.14,1468,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,0.0,4.33,0.21,13.30,13.33,13.39,13.51,13.40,3.05,3.01,3.02,3.08,1,2.06,2.05,2.03,2.06,2.07,2.12,...,21.18,20.98,21.20,21.09,0.22,1469,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,0.0,4.35,0.16,13.35,13.34,13.41,13.50,13.42,3.04,3.07,3.00,3.12,1,2.09,2.06,2.09,2.12,2.33,2.13,...,21.16,20.96,21.18,21.10,0.22,1469,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,TRAIN_39603,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,0.0,4.36,0.15,13.37,13.38,13.51,13.52,13.46,3.20,3.03,3.06,3.13,1,2.10,2.11,2.08,2.11,2.10,2.20,...,21.17,21.08,21.19,21.19,0.11,1469,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812,1.382,1.215,1.263,10.874,29.194,16.582,3.410,-26.486,-26.581,-22.772,24.261,-26.491,-26.584,-26.580
39603,TRAIN_39604,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,0.0,4.40,0.13,13.36,13.36,13.47,13.49,13.44,3.15,3.06,3.05,3.06,1,2.12,2.12,2.09,2.12,2.11,2.20,...,21.16,21.09,21.21,21.19,0.12,1458,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413,1.482,0.606,1.083,8.759,29.859,15.659,3.406,-27.308,-27.203,-24.674,23.427,-27.250,-27.334,-27.325
39604,TRAIN_39605,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,0.0,4.38,0.14,13.38,13.39,13.47,13.52,13.46,3.23,3.09,3.07,3.12,1,2.13,2.09,2.08,2.07,2.12,2.16,...,21.17,21.09,21.22,21.19,0.13,1459,1,1,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699,1.117,1.154,0.993,13.159,24.720,16.823,3.215,-26.502,-26.687,-22.577,24.301,-26.388,-26.425,-26.601
39605,TRAIN_39606,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,0.0,0.0,4.33,0.16,13.36,13.38,13.45,13.52,13.46,3.18,3.01,3.15,3.09,1,2.07,2.03,2.05,2.05,2.10,2.12,...,21.15,21.05,21.16,21.13,0.11,1469,1,1,62222.33,128.189679,121.4

In [ ]:
X_data = df.filter(regex='X') # Input : X Featrue
Y_data = df.filter(regex='Y') # Output : Y Feature

In [ ]:
cols = ["X_10","X_11"]
X_data[cols] = X_data[cols].replace(0, np.nan)

In [ ]:
# MICE 결측치 보간

imp = IterativeImputer(estimator = LinearRegression(), 
                       tol= 1e-10, 
                       max_iter=30, 
                       verbose=2,
                       min_value=0, 
                       imputation_order='roman')

X_data = pd.DataFrame(imp.fit_transform(X_data))

[IterativeImputer] Completing matrix with shape (39607, 56)
[IterativeImputer] Ending imputation round 1/30, elapsed time 13.82
[IterativeImputer] Change: 9.92197162406653, scaled tolerance: 1.1456363000000001e-05 
[IterativeImputer] Ending imputation round 2/30, elapsed time 26.80
[IterativeImputer] Change: 3.581745086478504, scaled tolerance: 1.1456363000000001e-05 
[IterativeImputer] Ending imputation round 3/30, elapsed time 38.08
[IterativeImputer] Change: 0.11653651638593487, scaled tolerance: 1.1456363000000001e-05 
[IterativeImputer] Ending imputation round 4/30, elapsed time 44.40
[IterativeImputer] Change: 0.0014251706987735702, scaled tolerance: 1.1456363000000001e-05 
[IterativeImputer] Ending imputation round 5/30, elapsed time 50.79
[IterativeImputer] Change: 4.780373155099937e-05, scaled tolerance: 1.1456363000000001e-05 
[IterativeImputer] Ending imputation round 6/30, elapsed time 57.22
[IterativeImputer] Change: 1.6034547627441498e-06, scaled tolerance: 1.145636300000

In [ ]:
X_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
0,70.544,103.320,67.47,1.0,101.892,74.983,29.45,62.38,245.71,2.633086,0.735073,4.34,0.18,13.34,13.37,13.40,13.52,13.44,3.11,3.17,3.06,3.13,1.0,2.05,2.07,2.05,2.06,2.06,2.35,1.49,1.69,1.46,1.74,12.99,12.88,12.89,12.99,-16.41,-16.36,-17.03,21.20,20.99,21.28,21.09,0.29,1463.0,1.0,1.0,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1.0,101.944,72.943,28.73,61.23,233.61,2.731266,0.542032,4.38,0.18,13.33,13.33,13.46,13.51,13.42,2.97,3.11,2.91,3.20,1.0,2.10,2.10,2.10,2.10,2.17,2.28,1.49,1.67,1.45,1.66,12.92,12.87,12.89,12.93,-16.06,-16.11,-16.74,21.16,21.03,21.16,21.13,0.13,1463.0,1.0,1.0,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1.0,103.153,72.943,28.81,105.77,272.20,2.891078,0.441316,4.36,0.15,13.36,13.36,13.39,13.51,13.43,3.04,3.04,3.01,3.12,1.0,2.07,2.06,2.04,2.07,2.10,2.26,1.49,1.69,1.46,1.68,12.97,12.87,12.87,13.00,-16.16,-16.17,-16.76,21.13,21.03,21.17,21.12,0.14,1468.0,1.0,1.0,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1.0,101.971,77.022,28.92,115.21,255.36,3.083304,0.371247,4.33,0.21,13.30,13.33,13.39,13.51,13.40,3.05,3.01,3.02,3.08,1.0,2.06,2.05,2.03,2.06,2.07,2.12,1.47,1.68,1.47,1.68,12.91,12.97,12.99,12.92,-16.05,-16.03,-16.67,21.18,20.98,21.20,21.09,0.22,1469.0,1.0,1.0,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1.0,101.981,70.904,29.68,103.38,241.46,3.175523,0.596681,4.35,0.16,13.35,13.34,13.41,13.50,13.42,3.04,3.07,3.00,3.12,1.0,2.09,2.06,2.09,2.12,2.33,2.13,1.49,1.68,1.47,1.82,12.96,12.85,12.91,12.96,-16.25,-16.23,-16.85,21.16,20.96,21.18,21.10,0.22,1469.0,1.0,1.0,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1.0,103.150,66.825,30.20,77.83,298.05,3.103335,1.621324,4.36,0.15,13.37,13.38,13.51,13.52,13.46,3.20,3.03,3.06,3.13,1.0,2.10,2.11,2.08,2.11,2.10,2.20,1.37,1.60,1.36,1.67,12.89,13.00,12.94,12.90,-16.09,-16.15,-16.79,21.17,21.08,21.19,21.19,0.11,1469.0,1.0,1.0,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812
39603,66.465,103.321,62.77,1.0,102.021,66.825,29.21,102.25,270.67,2.637958,0.998810,4.40,0.13,13.36,13.36,13.47,13.49,13.44,3.15,3.06,3.05,3.06,1.0,2.12,2.12,2.09,2.12,2.11,2.20,1.40,1.68,1.37,1.77,12.88,12.88,12.89,12.86,-15.70,-15.74,-16.40,21.16,21.09,21.21,21.19,0.12,1458.0,1.0,1.0,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413
39604,68.504,103.320,64.67,1.0,103.144,68.864,29.96,102.61,198.07,3.321680,0.824077,4.38,0.14,13.38,13.39,13.47,13.52,13.46,3.23,3.09,3.07,3.12,1.0,2.13,2.09,2.08,2.07,2.12,2.16,1.39,1.61,1.37,1.58,12.98,12.98,12.98,12.81,-16.12,-16.19,-16.80,21.17,21.09,21.22,21.19,0.13,1459.0,1.0,1.0,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699
39605,66.465,103.320,63.67,1.0,102.025,67.845,30.30,112.60,275.52,2.616090,1.299979,4.33,0.16,13.36,13.38,13.45,13.52,13.46,3.18,3.01,3.15,3.09,1.0,2.07,2.03,2.05,2.05,2.10,2.12,1.37,1.56,1.36,1.67,12.91,12.90,12.96,12.97,-16.36,-16.36,-16.97,21.15,21.05,21.16,21.13,0.11,1469.0,1.0,1.0,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929


In [ ]:
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install --user -U scikit-learn==0.23.2

Found existing installation: sklearn 0.0
Uninstalling sklearn-0.0:
  Successfully uninstalled sklearn-0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sklearn-0.0-py2.py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sklearn
sklearn.__version__

'0.23.2'

In [ ]:
!pip install pycaret[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numba-0.54.1-cp37-cp37m-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.3 MB)
  Using cached llvmlite-0.37.0-cp37-cp37m-manylinux2014_x86_64.whl (26.3 MB)
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.0
    Uninstalling llvmlite-0.39.0:
      Successfully uninstalled llvmlite-0.39.0
  Attempting uninstall: numba
    Found existing installation: numba 0.56.0
    Uninstalling numba-0.56.0:
      Successfully uninstalled numba-0.56.0


In [ ]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [ ]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


In [ ]:
!pip install category_encoders


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numba --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numba-0.56.0-cp37-cp37m-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.5 MB)
  Using cached llvmlite-0.39.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.6 MB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.37.0
    Uninstalling llvmlite-0.37.0:
      Successfully uninstalled llvmlite-0.37.0
  Attempting uninstall: numba
    Found existing installation: numba 0.54.1
    Uninstalling numba-0.54.1:
      Successfully uninstalled numba-0.54.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.10 requires numba<0.55, but you have numba 0.56.0 which is incompatible.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders.ordinal import OrdinalEncoder

In [ ]:
!pip install jupyter-client==6.1.12
!pip install tornado==5.1.0
!pip install fastai==2.0.15
!pip install fastai2==0.0.30
!pip install fastcore==1.0.16.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycaret.regression import *

In [ ]:
X_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
0,70.544,103.320,67.47,1.0,101.892,74.983,29.45,62.38,245.71,2.633086,0.735073,4.34,0.18,13.34,13.37,13.40,13.52,13.44,3.11,3.17,3.06,3.13,1.0,2.05,2.07,2.05,2.06,2.06,2.35,1.49,1.69,1.46,1.74,12.99,12.88,12.89,12.99,-16.41,-16.36,-17.03,21.20,20.99,21.28,21.09,0.29,1463.0,1.0,1.0,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1.0,101.944,72.943,28.73,61.23,233.61,2.731266,0.542032,4.38,0.18,13.33,13.33,13.46,13.51,13.42,2.97,3.11,2.91,3.20,1.0,2.10,2.10,2.10,2.10,2.17,2.28,1.49,1.67,1.45,1.66,12.92,12.87,12.89,12.93,-16.06,-16.11,-16.74,21.16,21.03,21.16,21.13,0.13,1463.0,1.0,1.0,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1.0,103.153,72.943,28.81,105.77,272.20,2.891078,0.441316,4.36,0.15,13.36,13.36,13.39,13.51,13.43,3.04,3.04,3.01,3.12,1.0,2.07,2.06,2.04,2.07,2.10,2.26,1.49,1.69,1.46,1.68,12.97,12.87,12.87,13.00,-16.16,-16.17,-16.76,21.13,21.03,21.17,21.12,0.14,1468.0,1.0,1.0,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1.0,101.971,77.022,28.92,115.21,255.36,3.083304,0.371247,4.33,0.21,13.30,13.33,13.39,13.51,13.40,3.05,3.01,3.02,3.08,1.0,2.06,2.05,2.03,2.06,2.07,2.12,1.47,1.68,1.47,1.68,12.91,12.97,12.99,12.92,-16.05,-16.03,-16.67,21.18,20.98,21.20,21.09,0.22,1469.0,1.0,1.0,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1.0,101.981,70.904,29.68,103.38,241.46,3.175523,0.596681,4.35,0.16,13.35,13.34,13.41,13.50,13.42,3.04,3.07,3.00,3.12,1.0,2.09,2.06,2.09,2.12,2.33,2.13,1.49,1.68,1.47,1.82,12.96,12.85,12.91,12.96,-16.25,-16.23,-16.85,21.16,20.96,21.18,21.10,0.22,1469.0,1.0,1.0,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1.0,103.150,66.825,30.20,77.83,298.05,3.103335,1.621324,4.36,0.15,13.37,13.38,13.51,13.52,13.46,3.20,3.03,3.06,3.13,1.0,2.10,2.11,2.08,2.11,2.10,2.20,1.37,1.60,1.36,1.67,12.89,13.00,12.94,12.90,-16.09,-16.15,-16.79,21.17,21.08,21.19,21.19,0.11,1469.0,1.0,1.0,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812
39603,66.465,103.321,62.77,1.0,102.021,66.825,29.21,102.25,270.67,2.637958,0.998810,4.40,0.13,13.36,13.36,13.47,13.49,13.44,3.15,3.06,3.05,3.06,1.0,2.12,2.12,2.09,2.12,2.11,2.20,1.40,1.68,1.37,1.77,12.88,12.88,12.89,12.86,-15.70,-15.74,-16.40,21.16,21.09,21.21,21.19,0.12,1458.0,1.0,1.0,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413
39604,68.504,103.320,64.67,1.0,103.144,68.864,29.96,102.61,198.07,3.321680,0.824077,4.38,0.14,13.38,13.39,13.47,13.52,13.46,3.23,3.09,3.07,3.12,1.0,2.13,2.09,2.08,2.07,2.12,2.16,1.39,1.61,1.37,1.58,12.98,12.98,12.98,12.81,-16.12,-16.19,-16.80,21.17,21.09,21.22,21.19,0.13,1459.0,1.0,1.0,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699
39605,66.465,103.320,63.67,1.0,102.025,67.845,30.30,112.60,275.52,2.616090,1.299979,4.33,0.16,13.36,13.38,13.45,13.52,13.46,3.18,3.01,3.15,3.09,1.0,2.07,2.03,2.05,2.05,2.10,2.12,1.37,1.56,1.36,1.67,12.91,12.90,12.96,12.97,-16.36,-16.36,-16.97,21.15,21.05,21.16,21.13,0.11,1469.0,1.0,1.0,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929


In [ ]:
Y_data["Y_01"]

0        2.056
1        1.446
2        1.251
3        1.464
4        0.983
         ...  
39602    1.382
39603    1.482
39604    1.117
39605    0.895
39606    1.147
Name: Y_01, Length: 39607, dtype: float64

In [ ]:
df = pd.concat([X_data, Y_data["Y_01"]], axis=1)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,Y_01
0,70.544,103.320,67.47,1.0,101.892,74.983,29.45,62.38,245.71,2.633086,0.735073,4.34,0.18,13.34,13.37,13.40,13.52,13.44,3.11,3.17,3.06,3.13,1.0,2.05,2.07,2.05,2.06,2.06,2.35,1.49,1.69,1.46,1.74,12.99,12.88,12.89,12.99,-16.41,-16.36,-17.03,21.20,20.99,21.28,21.09,0.29,1463.0,1.0,1.0,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256,2.056
1,69.524,103.321,65.17,1.0,101.944,72.943,28.73,61.23,233.61,2.731266,0.542032,4.38,0.18,13.33,13.33,13.46,13.51,13.42,2.97,3.11,2.91,3.20,1.0,2.10,2.10,2.10,2.10,2.17,2.28,1.49,1.67,1.45,1.66,12.92,12.87,12.89,12.93,-16.06,-16.11,-16.74,21.16,21.03,21.16,21.13,0.13,1463.0,1.0,1.0,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308,1.446
2,72.583,103.320,64.07,1.0,103.153,72.943,28.81,105.77,272.20,2.891078,0.441316,4.36,0.15,13.36,13.36,13.39,13.51,13.43,3.04,3.04,3.01,3.12,1.0,2.07,2.06,2.04,2.07,2.10,2.26,1.49,1.69,1.46,1.68,12.97,12.87,12.87,13.00,-16.16,-16.17,-16.76,21.13,21.03,21.17,21.12,0.14,1468.0,1.0,1.0,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232,1.251
3,71.563,103.320,67.57,1.0,101.971,77.022,28.92,115.21,255.36,3.083304,0.371247,4.33,0.21,13.30,13.33,13.39,13.51,13.40,3.05,3.01,3.02,3.08,1.0,2.06,2.05,2.03,2.06,2.07,2.12,1.47,1.68,1.47,1.68,12.91,12.97,12.99,12.92,-16.05,-16.03,-16.67,21.18,20.98,21.20,21.09,0.22,1469.0,1.0,1.0,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225,1.464
4,69.524,103.320,63.57,1.0,101.981,70.904,29.68,103.38,241.46,3.175523,0.596681,4.35,0.16,13.35,13.34,13.41,13.50,13.42,3.04,3.07,3.00,3.12,1.0,2.09,2.06,2.09,2.12,2.33,2.13,1.49,1.68,1.47,1.82,12.96,12.85,12.91,12.96,-16.25,-16.23,-16.85,21.16,20.96,21.18,21.10,0.22,1469.0,1.0,1.0,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762,0.983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1.0,103.150,66.825,30.20,77.83,298.05,3.103335,1.621324,4.36,0.15,13.37,13.38,13.51,13.52,13.46,3.20,3.03,3.06,3.13,1.0,2.10,2.11,2.08,2.11,2.10,2.20,1.37,1.60,1.36,1.67,12.89,13.00,12.94,12.90,-16.09,-16.15,-16.79,21.17,21.08,21.19,21.19,0.11,1469.0,1.0,1.0,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812,1.382
39603,66.465,103.321,62.77,1.0,102.021,66.825,29.21,102.25,270.67,2.637958,0.998810,4.40,0.13,13.36,13.36,13.47,13.49,13.44,3.15,3.06,3.05,3.06,1.0,2.12,2.12,2.09,2.12,2.11,2.20,1.40,1.68,1.37,1.77,12.88,12.88,12.89,12.86,-15.70,-15.74,-16.40,21.16,21.09,21.21,21.19,0.12,1458.0,1.0,1.0,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413,1.482
39604,68.504,103.320,64.67,1.0,103.144,68.864,29.96,102.61,198.07,3.321680,0.824077,4.38,0.14,13.38,13.39,13.47,13.52,13.46,3.23,3.09,3.07,3.12,1.0,2.13,2.09,2.08,2.07,2.12,2.16,1.39,1.61,1.37,1.58,12.98,12.98,12.98,12.81,-16.12,-16.19,-16.80,21.17,21.09,21.22,21.19,0.13,1459.0,1.0,1.0,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699,1.117
39605,66.465,103.320,63.67,1.0,102.025,67.845,30.30,112.60,275.52,2.616090,1.299979,4.33,0.16,13.36,13.38,13.45,13.52,13.46,3.18,3.01,3.15,3.09,1.0,2.07,2.03,2.05,2.05,2.10,2.12,1.37,1.56,1.36,1.67,12.91,12.90,12.96,12.97,-16.36,-16.36,-16.97,21.15,21.05,21.16,21.13,0.11,1469.0,1.0,1.0,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929,0.895


In [ ]:
model = setup(
    data = df,
    target = "Y_01",
    fold = 10
)

,Description,Value
0,session_id,1974
1,Target,Y_01
2,Original Data,"(39607, 57)"
3,Missing Values,False
4,Numeric Features,55
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(27724, 56)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='Y_01',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='m...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

In [ ]:
top5_model = compare_models(
               round=4,
               sort="RMSE",
               n_select = 5 )

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.2693,1.220000e-01,0.3493,4.640000e-02,0.1531,0.2823,0.883
gbr,Gradient Boosting Regressor,0.2695,1.224000e-01,0.3498,4.330000e-02,0.1534,0.2852,16.574
rf,Random Forest Regressor,0.2697,1.225000e-01,0.3499,4.260000e-02,0.1533,0.2814,74.210
catboost,CatBoost Regressor,0.2701,1.226000e-01,0.3501,4.190000e-02,0.1534,0.2832,15.997
et,Extra Trees Regressor,0.2698,1.228000e-01,0.3504,4.010000e-02,0.1536,0.2852,31.391
lr,Linear Regression,0.2720,1.242000e-01,0.3524,2.920000e-02,0.1546,0.2893,0.453
ridge,Ridge Regression,0.2720,1.243000e-01,0.3525,2.850000e-02,0.1547,0.2895,0.033
br,Bayesian Ridge,0.2721,1.245000e-01,0.3528,2.720000e-02,0.1548,0.2898,0.126
omp,Orthogonal Matching Pursuit,0.2739,1.257000e-01,0.3546,1.730000e-02,0.1556,0.2916,0.026
lasso,Lasso Regression,0.2761,1.279000e-01,0.3575,7.000000e-04,0.1568,0.2932,0.035


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 2
INFO:logs:[LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=1974, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_e

In [ ]:
blender_top5 = stack_models(estimator_list=top5_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2642,0.1167,0.3416,0.0608,0.1501,0.2469
1,0.2698,0.1236,0.3515,0.0504,0.1559,0.3344
2,0.2685,0.1205,0.3471,0.0593,0.1526,0.2940
3,0.2684,0.1204,0.3469,0.0671,0.1536,0.2707
4,0.2618,0.1165,0.3413,0.0649,0.1485,0.2819
5,0.2711,0.1239,0.3520,0.0395,0.1551,0.3008
6,0.2684,0.1198,0.3462,0.0425,0.1505,0.2688
7,0.2716,0.1269,0.3562,0.0643,0.1561,0.3228
8,0.2692,0.1224,0.3499,0.0470,0.1523,0.2604


INFO:logs:create_model_container: 21
INFO:logs:master_model_container: 21
INFO:logs:display_container: 3
INFO:logs:StackingRegressor(cv=5,
                  estimators=[('lightgbm',
                               LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=1974, reg_alpha=0.0,
                                          

In [ ]:
final_model = finalize_model(blender_top5)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=StackingRegressor(cv=5,
                  estimators=[('lightgbm',
                               LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=1974, reg_alpha=0.0,
                                             reg_lambda=0.0, silent='...
         

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/DACON/LG/test.csv")

In [ ]:
prediction_test = predict_model(final_model, data=test)
pred_bin=prediction_test["Label"].tolist()

In [ ]:
df = pd.DataFrame(pred_bin)
df.to_csv("Y1.csv")